In [1]:
import pandas as pd # import pandas library

In [2]:
df = pd.read_csv('ViewingActivity-sample.csv') # read our Netflix data csv into a pandas dataframe: df

In [3]:
df.shape # gives # of rows and columns in df

(200, 10)

In [4]:
df.head() # gives the first few rows of df

,Profile Name,Start Time,Duration,Attributes,Title,Supplemental Video Type,Device Type,Bookmark,Latest Bookmark,Country
0,Charlie,2013-03-20 5:17:53,0:00:05,NaN,Star Trek: Deep Space Nine: Season 5: Empok No...,NaN,Mac,0:00:05,Not latest view,US (United States)
1,Charlie,2013-03-20 4:27:45,0:44:31,NaN,Star Trek: Deep Space Nine: Season 5: Blaze of...,NaN,Mac,0:44:31,Not latest view,US (United States)
2,Charlie,2013-03-20 4:05:21,0:22:06,NaN,Star Trek: Deep Space Nine: Season 5: Children...,NaN,Mac,0:44:37,0:44:37,US (United States)
3,Charlie,2013-03-20 0:20:03,0:48:14,NaN,The Invisible War,NaN,Microsoft Xbox 360,0:53:18,0:53:18,US (United States)
4,Charlie,2013-03-20 0:10:31,0:04:51,NaN,The Invisible War,NaN,Mac,0:05:01,Not latest view,US (United States)


In [5]:
# cleaning the data
# drop unnecessary columns
df.drop(['Profile Name', 'Attributes', 'Supplemental Video Type', 
         'Device Type', 'Bookmark', 'Latest Bookmark', 'Country'], axis=1, inplace=True)

In [6]:
df.dtypes # gives list of datatypes for each column in df
          # 'object' datatype means they're strings

Start Time    object
Duration      object
Title         object
dtype: object

In [7]:
df['Start Time'] = pd.to_datetime(df['Start Time'], utc=True) # change datatype of 'Start Time' to datetime with timezone=utc
df.dtypes

Start Time    datetime64[ns, UTC]
Duration                   object
Title                      object
dtype: object

In [8]:
# changing timezone of 'Start Time' from 'UTC' to 'US/Eastern'
df.set_index('Start Time', inplace=True) # change 'Start Time' column into df's index
df.index = df.index.tz_convert('US/Eastern') # convert timezone
df.reset_index(inplace=True) # reset the index so 'Start Time' becomes a column again

In [9]:
# changing datatype of 'Duration' column to 'timedelta' (measure of time duration that pandas understands)
df['Duration'] = pd.to_timedelta(df['Duration'])

In [10]:
df['Title'].unique() # analyzing the 'Title' column to make it make sense

array(['Star Trek: Deep Space Nine: Season 5: Empok Nor (Episode 24)',
       'Star Trek: Deep Space Nine: Season 5: Blaze of Glory (Episode 23)',
       'Star Trek: Deep Space Nine: Season 5: Children of Time (Episode 22)',
       'The Invisible War',
       'Star Trek: Deep Space Nine: Season 5: Soldiers of the Empire (Episode 21)',
       'Star Trek: Deep Space Nine: Season 5: Ferengi Love Songs (Episode 20)',
       'The Office (U.S.): Season 5: The Duel (Episode 12)',
       'The Office (U.S.): Season 5: Moroccan Christmas (Episode 11)',
       'The Office (U.S.): Season 5: The Surplus (Episode 10)',
       'The Office (U.S.): Season 5: Frame Toby (Episode 9)',
       'The Office (U.S.): Season 5: Business Trip (Episode 8)',
       'The Office (U.S.): Season 5: Customer Survey (Episode 7)',
       'The Office (U.S.): Season 5: Employee Transfer (Episode 6)',
       'The Office (U.S.): Season 5: Crime Aid (Episode 5)',
       'The Office (U.S.): Season 5: Baby Shower (Episode 4)',


In [11]:
# parsing 'Title' column into 'Name', 'Season', 'Episode Name', 'Episode #'
df[['Name', 'Season', 'Episode Name', 'Episode #']] = df['Title'].str.extract('^(.*?): Season (\d+): (.*) \(Episode (\d+)\)$', expand=True)
df

,Start Time,Duration,Title,Name,Season,Episode Name,Episode #
0,2013-03-20 01:17:53-04:00,0 days 00:00:05,Star Trek: Deep Space Nine: Season 5: Empok No...,Star Trek: Deep Space Nine,5,Empok Nor,24
1,2013-03-20 00:27:45-04:00,0 days 00:44:31,Star Trek: Deep Space Nine: Season 5: Blaze of...,Star Trek: Deep Space Nine,5,Blaze of Glory,23
2,2013-03-20 00:05:21-04:00,0 days 00:22:06,Star Trek: Deep Space Nine: Season 5: Children...,Star Trek: Deep Space Nine,5,Children of Time,22
3,2013-03-19 20:20:03-04:00,0 days 00:48:14,The Invisible War,NaN,NaN,NaN,NaN
4,2013-03-19 20:10:31-04:00,0 days 00:04:51,The Invisible War,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
195,2013-03-01 19:45:12-05:00,0 days 00:05:18,Star Trek: Deep Space Nine: Season 4: Body Par...,Star Trek: Deep Space Nine,4,Body Parts,24
196,2013-03-01 18:46:16-05:00,0 days 00:58:12,Louis C.K.: Live at the Beacon Theater,NaN,NaN,NaN,NaN
197,2013-03-01 18:21:19-05:00,0 days 00:21:59,Star Trek: Deep Space Nine: Season 4: Body Par...,Star Trek: Deep Space Nine,4,Body Parts,24
198,2013-03-01 17:36:48-05:00,0 days 00:44:26,Star Trek: Deep Space Nine: Season 4: The Quic...,Star Trek: Deep Space Nine,4,The Quickening,23


In [12]:
df['Name'].unique() # gives the list of unique names of movies/shows entries
                    # notice that 'nan' is there
                    # gotta look into that

array(['Star Trek: Deep Space Nine', nan, 'The Office (U.S.)', 'Archer',
       'Buffy the Vampire Slayer'], dtype=object)

In [13]:
df['Name'].isnull().sum() # gives the sum of null values in 'Name' column

5

In [14]:
df[df['Name'].isnull()] # gives the records where 'Name' is null
                        # turns out 'Title' names with no season or episode details (probably movies) did not get parsed

,Start Time,Duration,Title,Name,Season,Episode Name,Episode #
3,2013-03-19 20:20:03-04:00,0 days 00:48:14,The Invisible War,NaN,NaN,NaN,NaN
4,2013-03-19 20:10:31-04:00,0 days 00:04:51,The Invisible War,NaN,NaN,NaN,NaN
158,2013-03-07 23:31:28-05:00,0 days 00:00:16,30 for 30: The Two Escobars,NaN,NaN,NaN,NaN
173,2013-03-05 18:02:57-05:00,0 days 00:23:29,30 for 30: The Two Escobars,NaN,NaN,NaN,NaN
196,2013-03-01 18:46:16-05:00,0 days 00:58:12,Louis C.K.: Live at the Beacon Theater,NaN,NaN,NaN,NaN


In [15]:
df.dropna(subset = ['Name'], inplace=True) # drops the records where 'Name' is null
df.reset_index(drop=True, inplace=True) # resets index
df

,Start Time,Duration,Title,Name,Season,Episode Name,Episode #
0,2013-03-20 01:17:53-04:00,0 days 00:00:05,Star Trek: Deep Space Nine: Season 5: Empok No...,Star Trek: Deep Space Nine,5,Empok Nor,24
1,2013-03-20 00:27:45-04:00,0 days 00:44:31,Star Trek: Deep Space Nine: Season 5: Blaze of...,Star Trek: Deep Space Nine,5,Blaze of Glory,23
2,2013-03-20 00:05:21-04:00,0 days 00:22:06,Star Trek: Deep Space Nine: Season 5: Children...,Star Trek: Deep Space Nine,5,Children of Time,22
3,2013-03-19 18:34:08-04:00,0 days 00:22:29,Star Trek: Deep Space Nine: Season 5: Children...,Star Trek: Deep Space Nine,5,Children of Time,22
4,2013-03-19 01:07:46-04:00,0 days 00:33:10,Star Trek: Deep Space Nine: Season 5: Soldiers...,Star Trek: Deep Space Nine,5,Soldiers of the Empire,21
...,...,...,...,...,...,...,...
190,2013-03-02 18:33:55-05:00,0 days 00:17:17,Star Trek: Deep Space Nine: Season 4: Body Par...,Star Trek: Deep Space Nine,4,Body Parts,24
191,2013-03-01 19:45:12-05:00,0 days 00:05:18,Star Trek: Deep Space Nine: Season 4: Body Par...,Star Trek: Deep Space Nine,4,Body Parts,24
192,2013-03-01 18:21:19-05:00,0 days 00:21:59,Star Trek: Deep Space Nine: Season 4: Body Par...,Star Trek: Deep Space Nine,4,Body Parts,24
193,2013-03-01 17:36:48-05:00,0 days 00:44:26,Star Trek: Deep Space Nine: Season 4: The Quic...,Star Trek: Deep Space Nine,4,The Quickening,23


In [22]:
df.groupby('Name')['Duration'].sum()

Name
Archer                       0 days 03:15:21
Buffy the Vampire Slayer     0 days 00:50:01
Star Trek: Deep Space Nine   0 days 20:02:03
The Office (U.S.)            1 days 15:00:39
Name: Duration, dtype: timedelta64[ns]

In [21]:
df.groupby(['Name', 'Episode Name'])['Duration'].sum()

Name               Episode Name             
Archer             Blood Test                  0 days 00:11:45
                   El Secuestro                0 days 00:00:14
                   Heart of Archness, Part 1   0 days 00:21:20
                   Heart of Archness, Part 2   0 days 00:19:25
                   Heart of Archness, Part 3   0 days 00:19:30
                                                     ...      
The Office (U.S.)  Viewing Party               0 days 00:21:31
                   WUPHF.com                   0 days 00:21:33
                   Weight Loss: Part 1         0 days 00:21:02
                   Weight Loss: Part 2         0 days 00:21:10
                   Women's Appreciation        0 days 00:28:35
Name: Duration, Length: 143, dtype: timedelta64[ns]

In [24]:
df['Weekday'] = df['Start Time'].dt.weekday
df['Hour'] = df['Start Time'].dt.hour
df

,Start Time,Duration,Title,Name,Season,Episode Name,Episode #,Weekday,Hour
0,2013-03-20 01:17:53-04:00,0 days 00:00:05,Star Trek: Deep Space Nine: Season 5: Empok No...,Star Trek: Deep Space Nine,5,Empok Nor,24,2,1
1,2013-03-20 00:27:45-04:00,0 days 00:44:31,Star Trek: Deep Space Nine: Season 5: Blaze of...,Star Trek: Deep Space Nine,5,Blaze of Glory,23,2,0
2,2013-03-20 00:05:21-04:00,0 days 00:22:06,Star Trek: Deep Space Nine: Season 5: Children...,Star Trek: Deep Space Nine,5,Children of Time,22,2,0
3,2013-03-19 18:34:08-04:00,0 days 00:22:29,Star Trek: Deep Space Nine: Season 5: Children...,Star Trek: Deep Space Nine,5,Children of Time,22,1,18
4,2013-03-19 01:07:46-04:00,0 days 00:33:10,Star Trek: Deep Space Nine: Season 5: Soldiers...,Star Trek: Deep Space Nine,5,Soldiers of the Empire,21,1,1
...,...,...,...,...,...,...,...,...,...
190,2013-03-02 18:33:55-05:00,0 days 00:17:17,Star Trek: Deep Space Nine: Season 4: Body Par...,Star Trek: Deep Space Nine,4,Body Parts,24,5,18
191,2013-03-01 19:45:12-05:00,0 days 00:05:18,Star Trek: Deep Space Nine: Season 4: Body Par...,Star Trek: Deep Space Nine,4,Body Parts,24,4,19
192,2013-03-01 18:21:19-05:00,0 days 00:21:59,Star Trek: Deep Space Nine: Season 4: Body Par...,Star Trek: Deep Space Nine,4,Body Parts,24,4,18
193,2013-03-01 17:36:48-05:00,0 days 00:44:26,Star Trek: Deep Space Nine: Season 4: The Quic...,Star Trek: Deep Space Nine,4,The Quickening,23,4,17


In [34]:
df.groupby('Weekday')['Duration'].sum()

Weekday
0   0 days 13:55:01
1   0 days 06:07:17
2   0 days 10:31:22
3   0 days 03:47:30
4   0 days 08:48:26
5   0 days 10:56:55
6   0 days 09:01:33
Name: Duration, dtype: timedelta64[ns]

In [32]:
df.groupby('Hour').size()

Hour
0     19
1     24
2      7
3      2
10     3
11     1
12     4
13     6
14     8
15    10
16     9
17    12
18    18
19    23
20    14
21    15
22     8
23    12
dtype: int64